In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""
import tensorflow as tf
import numpy as np

2023-10-18 17:02:38.700870: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 17:02:41.337450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [29]:
k = 3  # 8
e = 7  # 32
b = 5  # 64
n = 128 # 750
d = 33  # 206
v = 8  # 16

In [3]:
q_layer = tf.keras.layers.Dense(k*e, activation='relu')

In [10]:
x = np.random.randn(b, n, d)
x.shape, np.prod(x.shape)

((5, 128, 33), 21120)

In [11]:
q = q_layer(x)
q = tf.reshape(q, (b, n, k, e))
q

<tf.Tensor: shape=(5, 128, 3, 7), dtype=float32, numpy=
array([[[[0.        , 0.        , 0.        , ..., 0.8519672 ,
          0.5698146 , 0.3554971 ],
         [0.        , 0.1651705 , 0.        , ..., 0.06637713,
          0.        , 0.12587261],
         [0.        , 0.        , 0.38963243, ..., 1.2807512 ,
          0.        , 0.        ]],

        [[0.        , 1.1355458 , 0.1658106 , ..., 0.        ,
          0.        , 0.        ],
         [0.7387805 , 0.        , 0.        , ..., 0.14723931,
          0.        , 1.3055836 ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]],

        [[0.        , 1.0450399 , 0.3290836 , ..., 0.        ,
          1.1476657 , 0.        ],
         [0.        , 0.        , 0.14164874, ..., 0.7693904 ,
          0.49279934, 0.        ],
         [0.00821878, 0.        , 0.03284475, ..., 0.        ,
          0.67159075, 0.04966677]],

        ...,

        [[0.        , 1.5003881 , 0.      

In [12]:
k_layer = tf.keras.layers.Dense(k*e, activation='relu')

In [13]:
k_tensor = k_layer(x)
k_tensor = tf.reshape(k_tensor, (b, n, k, e))
k_tensor

<tf.Tensor: shape=(5, 128, 3, 7), dtype=float32, numpy=
array([[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 4.68240082e-01],
         [0.00000000e+00, 1.93117285e+00, 6.90380096e-01, ...,
          0.00000000e+00, 4.98773932e-01, 7.09462881e-01],
         [5.80381215e-01, 8.23017955e-01, 1.47447063e-04, ...,
          1.42311740e+00, 0.00000000e+00, 0.00000000e+00]],

        [[1.60682249e+00, 0.00000000e+00, 2.63139486e-01, ...,
          4.32512343e-01, 1.25747108e+00, 0.00000000e+00],
         [0.00000000e+00, 5.97176313e-01, 0.00000000e+00, ...,
          1.84754205e+00, 7.03298330e-01, 0.00000000e+00],
         [5.97413599e-01, 2.95061231e-01, 0.00000000e+00, ...,
          0.00000000e+00, 2.78401881e-01, 0.00000000e+00]],

        [[4.29811984e-01, 0.00000000e+00, 0.00000000e+00, ...,
          3.66450608e-01, 1.05187404e+00, 4.52554643e-01],
         [3.69707905e-02, 0.00000000e+00, 8.06317255e-02, ...,
          0.00000000e+0

In [24]:
m = tf.reduce_sum(q*k_tensor, axis=3)

In [26]:
softmax = tf.keras.layers.Softmax(axis=-1)

In [27]:
s = softmax(m)
s

<tf.Tensor: shape=(5, 128, 3), dtype=float32, numpy=
array([[[0.13309851, 0.16950904, 0.69739246],
        [0.31114903, 0.3909859 , 0.2978651 ],
        [0.14573221, 0.7739708 , 0.08029696],
        ...,
        [0.46794662, 0.24884936, 0.28320414],
        [0.0705217 , 0.8937356 , 0.03574271],
        [0.08626413, 0.19667253, 0.71706325]],

       [[0.00109076, 0.00125865, 0.99765056],
        [0.082483  , 0.43096524, 0.4865517 ],
        [0.13087903, 0.178657  , 0.6904639 ],
        ...,
        [0.22875328, 0.128972  , 0.6422747 ],
        [0.82840884, 0.0606955 , 0.11089568],
        [0.02249251, 0.04863402, 0.9288735 ]],

       [[0.8568019 , 0.07964583, 0.06355219],
        [0.61824626, 0.09702252, 0.2847312 ],
        [0.9774252 , 0.00897976, 0.01359515],
        ...,
        [0.06746736, 0.665072  , 0.2674606 ],
        [0.18392175, 0.46078286, 0.3552954 ],
        [0.5129568 , 0.13257785, 0.35446525]],

       [[0.46380532, 0.46213955, 0.07405519],
        [0.03488824, 0.92180

In [28]:
s[0, 0]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.13309851, 0.16950904, 0.69739246], dtype=float32)>

In [30]:
v_layer = tf.keras.layers.Dense(k*v, activation='relu')

In [31]:
v_tensor = v_layer(x)
v_tensor = tf.reshape(v_tensor, (b, n, k, v))
v_tensor

<tf.Tensor: shape=(5, 128, 3, 8), dtype=float32, numpy=
array([[[[8.41866255e-01, 1.16527975e-01, 4.92120624e-01, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 1.95927814e-01, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 5.15190542e-01, ...,
          0.00000000e+00, 0.00000000e+00, 1.02623665e+00]],

        [[9.45346832e-01, 0.00000000e+00, 1.79677355e+00, ...,
          1.33384675e-01, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 1.38714957e+00, 0.00000000e+00, ...,
          1.90986907e+00, 1.69792759e+00, 1.11089182e+00],
         [0.00000000e+00, 1.21318460e+00, 5.28670192e-01, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

        [[0.00000000e+00, 0.00000000e+00, 7.82751620e-01, ...,
          2.24770755e-01, 0.00000000e+00, 7.84755468e-01],
         [2.13158298e+00, 0.00000000e+00, 5.98843433e-02, ...,
          5.30625343e-0

In [34]:
o = v_tensor*tf.expand_dims(s, axis=-1)
o = tf.reduce_sum(o, axis=2)
o

<tf.Tensor: shape=(5, 128, 8), dtype=float32, numpy=
array([[[1.12051144e-01, 1.55097004e-02, 4.58002061e-01, ...,
         0.00000000e+00, 0.00000000e+00, 7.15689719e-01],
        [2.94143736e-01, 9.03721333e-01, 7.16536760e-01, ...,
         7.88234413e-01, 6.63865745e-01, 4.34343040e-01],
        [1.64978290e+00, 0.00000000e+00, 1.60420850e-01, ...,
         4.43444848e-01, 4.30640936e-01, 4.33451474e-01],
        ...,
        [1.00388527e+00, 4.28788543e-01, 5.46316624e-01, ...,
         6.88233733e-01, 4.40684378e-01, 1.18466055e+00],
        [0.00000000e+00, 4.30591106e-02, 1.84863269e+00, ...,
         0.00000000e+00, 3.66431832e-01, 2.57575251e-02],
        [0.00000000e+00, 1.00251831e-01, 2.20018327e-01, ...,
         3.43543738e-02, 7.76060969e-02, 4.38687205e-02]],

       [[3.74257654e-01, 0.00000000e+00, 2.86451890e-03, ...,
         1.05425704e-03, 8.69973361e-01, 7.46727106e-04],
        [1.33413434e+00, 4.89779413e-01, 3.14888328e-01, ...,
         1.21045804e+00, 5.716

In [35]:
206*128

26368